In [ ]:
import pandas as pd
from urllib.request import urlopen
import json
from tensorflow import keras
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler

### Get historical data from the NASA POWER PROJECT API
##### Data is collected from: 1st Jan 2012 through 19th March 2022

In [ ]:
url = "https://power.larc.nasa.gov/api/temporal/daily/point?start=20120101&end=20220319&latitude=-1.5177&longitude=37.2634&parameters=T2M,PS,WS10M,QV2M,PRECTOTCORR&community=AG&format=csv"

csv_path = keras.utils.get_file(fname="machakos-county-2012-2022.csv", origin=url)

In [ ]:
def parse_date(x):
    return datetime.strptime(x, '%Y %j')

In [ ]:
# Skip the CSV description rows
df = pd.read_csv(csv_path, skiprows=13, parse_dates={'date': ['YEAR', 'DOY']}, date_parser=parse_date, skipinitialspace=True, index_col=0)

In [ ]:
df.head()

In [ ]:
# Get column names
df.columns

In [ ]:
# Remove empty values
indexes_to_drop = df.index[df['T2M'] == -999.00]
df.drop(indexes_to_drop, inplace=True)

In [ ]:
df.info()

In [ ]:
# Box plot
df.plot.box()

# plt.savefig("Box plot", dpi=1200)


In [ ]:
# Visualize trends
df.plot.line(figsize=(12, 4), subplots=True)
# plt.savefig("Sub plot", dpi=1200)

In [ ]:
def show_heatmap(data):
    plt.matshow(data.corr())
    plt.xticks(range(data.shape[1]), data.columns, fontsize=14, rotation=90)
    plt.gca().xaxis.tick_bottom()
    plt.yticks(range(data.shape[1]), data.columns, fontsize=14)

    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title("Feature correlation heatmap", fontsize=14)
    plt.show()

show_heatmap(df)

In [ ]:
correlation = df.corr()
print(correlation['PRECTOTCORR'].sort_values(ascending = False),'\n')

In [ ]:
for i in ['WS10M', 'T2M', 'PS', 'QV2M']:
    df.plot.scatter(x=i, y='PRECTOTCORR')

In [ ]:
# Drop PS columns as it is multicorrelated to the T2M column
df.drop(['PS'], axis=1, inplace=True)

In [ ]:
# LSTM uses sigmooid and tanh which is sensitive to magnitude hence must be standardized 
scaler = StandardScaler()
df_train_scaled = scaler.fit_transform(df)

In [ ]:

# Multivariate output data preparation
from numpy import array
from numpy import hstack

In [ ]:
# split multivariate sequence into samples
def split_sequences(sequences, n_steps_back, n_steps_future):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of the pattern
        end_index = i + n_steps_back
        out_end_index = end_index + n_steps_future - 1
        # check if index is out of bound
        if out_end_index > len(sequences) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_index, :-1], sequences[end_index-1:out_end_index, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


### Problem Framing
#### Given the past 28 days historical weather data, forecast the rainfall for the next 7 days

In [ ]:
# select number of time steps
n_steps_back, n_steps_future = 28, 7
# convert into input/output
X, y = split_sequences(df_train_scaled, n_steps_back, n_steps_future)

In [ ]:
# Split data into training and validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(y_train.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# fit the model
history = model.fit(X_train, y_train, epochs=50, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)

plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Test')
plt.legend()

In [ ]:
# Make a prediction
yhat = model.predict(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2])
# Invert scaling for forecast
inv_yhat = np.concatenate((yhat, X_test[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# Invert scaling for actual
y_test = y_test.reshape((len(y_test), 1))
inv_y = np.concatenate((y_test, X_test[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
# Calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))

## Evaluation Metric
Evaluate each time step separately in order to:
* Comment on the skill at a particular lead time (1 day vs 3 day)
* Contrast models based on their skills at different lead times (models good at +1 day vs models good at days +5)

Adopt an error metric that has the metric in (mm) similar to our target variable, rainfall.
Use Root Mean Square Error (RMSE), it has a higher punishment on forecast errors.